In [2]:
%matplotlib inline
import torch
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import random
import os
from PIL import Image
import matplotlib.pyplot as plt

In [52]:
x = random.randint(0,1) 
x

1

In [4]:
class TrafficIntersectionDataset(Dataset):
    
    def __init__(self,imageFolderDataset,transform=None,should_invert=True):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        self.should_invert = should_invert
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)
        #we need to make sure approx 50% of images are in the same class
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                #keep looping till the same class image is found
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1]==img1_tuple[1]:
                    break
        else:
            img1_tuple = random.choice(self.imageFolderDataset.imgs)

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
        img0 = img0.convert("L")
        img1 = img1.convert("L")
        
        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)
            img1 = PIL.ImageOps.invert(img1)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1 , torch.from_numpy(np.array([int(img1_tuple[1]!=img0_tuple[1])],dtype=np.float32))
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [65]:
mg0 = random.choice(folder_dataset.imgs)
mg0

('D:/Projects/Tracking Project/Datasets/UA-Detrac Dataset/Insight-MVT_Annotation_Train\\MVI_20012\\img00074.jpg',
 1)

In [63]:
class Config():
    training_dir = "D:/Projects/Tracking Project/Datasets/UA-Detrac Dataset/Insight-MVT_Annotation_Train"
    testing_dir = "D:/Projects/Tracking Project/Datasets/UA-Detrac Dataset/Insight-MVT_Annotation_Test"
    train_batch_size = 64
    train_number_epochs = 100

In [54]:
folder_dataset = dset.ImageFolder(root=Config.training_dir)

In [12]:
siamese_traffic_dataset = TrafficIntersectionDataset(imageFolderDataset=folder_dataset,
                                        transform=transforms.Compose([transforms.Resize((100,100)),
                                                                      transforms.ToTensor()
                                                                      ])
                                       ,should_invert=False)

In [33]:
for i in range(len(siamese_traffic_dataset)):
    sample = siamese_traffic_dataset[i]

In [35]:
sample

(tensor([[[0.2157, 0.2039, 0.2196,  ..., 0.4667, 0.4275, 0.3922],
          [0.2039, 0.2157, 0.2275,  ..., 0.4549, 0.3922, 0.3686],
          [0.2078, 0.2353, 0.2471,  ..., 0.4353, 0.3686, 0.3647],
          ...,
          [0.2941, 0.2627, 0.2941,  ..., 0.4510, 0.5020, 0.5412],
          [0.2784, 0.2510, 0.3098,  ..., 0.4784, 0.5020, 0.5098],
          [0.2784, 0.2510, 0.3059,  ..., 0.4667, 0.5020, 0.4941]]]),
 tensor([[[0.2078, 0.2275, 0.2275,  ..., 0.4588, 0.4314, 0.4157],
          [0.2000, 0.2353, 0.2078,  ..., 0.4588, 0.4471, 0.4235],
          [0.2078, 0.2196, 0.2039,  ..., 0.4471, 0.4471, 0.4235],
          ...,
          [0.3020, 0.2627, 0.3137,  ..., 0.6941, 0.7529, 0.7686],
          [0.3176, 0.2627, 0.3020,  ..., 0.5333, 0.5961, 0.6235],
          [0.3216, 0.2588, 0.2980,  ..., 0.4157, 0.4118, 0.4471]]]),
 tensor([0.]))